# Demo of Network Analysis using Networkx

In [86]:
import pandas as pd
import networkx as nx
from bokeh.io import output_file, show,output_notebook, output_file
from bokeh.models import (BoxZoomTool, Circle, HoverTool,
                                  MultiLine, Plot, Range1d, ResetTool,)
from bokeh.palettes import Spectral4
from bokeh.plotting import from_networkx
from bokeh.transform import transform    
from bokeh.models import CustomJSTransform, LabelSet
from csv import reader
output_notebook()

Loading BokehJS ...

In [87]:
graph = []
# import nodes and edges into pandas
nodes = pd.read_csv ('Nodes.csv')
nodes.set_index("id", inplace=True)
edges = pd.read_csv ('Edges.csv')
# add edges to graph
for i in range(0,len(edges)):
    source = edges.iloc[i].Source
    sourcename = nodes.loc[source].Label
    target = edges.iloc[i].Target
    targetname = nodes.loc[target].Label
    graph.append((sourcename,targetname))
G = nx.Graph()
G.add_edges_from(graph)
edge_attrs = {}

# Show with Bokeh
plot = Plot(plot_width=1000, plot_height=1000,
            x_range=Range1d(-1.1, 1.1), y_range=Range1d(-1.1, 1.1))
plot.title.text = " Network Graph"


graph_renderer = from_networkx(G, nx.shell_layout, scale=0.8, center=(0, 0))

graph_renderer.node_renderer.glyph = Circle(size=15, fill_color=Spectral4[0])
graph_renderer.edge_renderer.glyph = MultiLine( line_alpha=0.8, line_width=1)
source = graph_renderer.node_renderer.data_source
source.data['names'] = source.data['index']
plot.renderers.append(graph_renderer)

# create a transform that can extract the actual x,y positions
code = """
    var result = new Float64Array(xs.length)
    for (var i = 0; i < xs.length; i++) {
        result[i] = provider.graph_layout[xs[i]][%s]
    }
    return result
"""
xcoord = CustomJSTransform(v_func=code % "0", args=dict(provider=graph_renderer.layout_provider))
ycoord = CustomJSTransform(v_func=code % "1", args=dict(provider=graph_renderer.layout_provider))

# Use the transforms to supply coords to a LabelSet 
labels = LabelSet(x=transform('index', xcoord),
                  y=transform('index', ycoord),
                  text='names', text_font_size="12px",
                  x_offset=5, y_offset=5,
                  source=source, render_mode='canvas')

plot.add_layout(labels)
output_file("NetworkAnalysis.html")
show(plot)

In [ ]:
nx.degree_centrality(G) 

In [ ]:
nx.betweenness_centrality(G)

In [ ]:
nx.closeness_centrality(G)

In [ ]:
list(nx.connected_components(G))

In [ ]:
nx.density(G)


In [ ]:
nx.diameter(G)